#### 透过经纬度寻找所属省份，城市和区域
源代码地址: 
https://www.cnblogs.com/shixinzei/p/11029279.html

https://www.cnblogs.com/zlc364624/p/12455791.html

（若高德地图如下）https://blog.csdn.net/weixin_42473844/article/details/100163838

1. 需调用百度地图API实现得申请百度账号或者登陆百度账号，然后申请自己的ak秘钥。链接如下：http://lbsyun.baidu.com/apiconsole/key?application=key

2. 逆地理编码了解服务文档：http://lbsyun.baidu.com/index.php?title=webapi/guide/webservice-geocoding-abroad

In [1]:
#encoding=utf8
import json
import urllib.request
import openpyxl
import xlwt
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
# 设定参数; Set your parameters
ak_key = '你的百度api ak key' #你的百度api ak key
df=pd.read_excel('.\TestLocation.xlsx')
col_lng_lat = ['经度','纬度'] # LNG:经度; LAT:纬度

In [3]:
#基于百度地图API下的经纬度信息来解析地理位置信息
def getlocation(lat,lng,ak_key):
    ak_key = 'lLoGaQmqyIBLGibfmVP9eZRiDGEsseth'
    url = 'http://api.map.baidu.com/reverse_geocoding/v3/?ak=' + ak_key + '&output=json&coordtype=wgs84ll&location=' + lat + ',' + lng
    print('Retrieving: ', url)
    req = urllib.request.urlopen(url)  # json格式的返回数据
    data = req.read().decode("utf-8")  # 将其他编码的字符串解码成unicode
    # 解析数据
    try:
        js = json.loads(data)
    except:
        js = None
    return js

In [4]:
def jsonFormat(lat,lng,ak_key):
    js = getlocation(lat,lng,ak_key)
    
    if not js or 'status' not in js or js['status'] != 0:
        print('======Failure====')
        print('Cannot obtain address 经度：', lat, '； 纬度: ', lng)
        dictjson={}#声明一个字典
        return dictjson
    
    dictjson={}#声明一个字典
    #get()获取json里面的数据
    jsonResult = js.get('result')
    # 百度地图辨认的经纬度
    location = jsonResult.get('location')
    lng_baidu = location.get('lng')
    lat_baidu = location.get('lat')
    # 完整地址
    formatted_address = jsonResult.get('formatted_address')
    # 经营单位
    business = jsonResult.get('business')
    # 地址元素
    address = jsonResult.get('addressComponent')
    #国家
    country = address.get('country')
    #国家编号（0：中国）
    country_code = address.get('country_code')
    #国家编号_英文简称_三位（CHN：中国）
    country_code_iso = address.get('country_code_iso')
    #国家编号_英文简称_二位（CN：中国）
    country_code_iso2 = address.get('country_code_iso2')
    #省
    province = address.get('province')
    #城市
    city = address.get('city')
    #城市等级
    city_level = address.get('city_level')
    #区/县
    district = address.get('district')
    #街道/镇
    town = address.get('town')
    town_code = address.get('town_code')
    #区划代码
    adcode = address.get('adcode')
    # 街道
    street=address.get('street')
    # 街道编号
    street_number=address.get('street_number')
    
    #把获取到的值，添加到字典里（添加）
    dictjson['formatted_address'] = formatted_address
#     dictjson['business'] = business
#     dictjson['country'] = country
#     dictjson['country_code'] = country_code
    dictjson['province'] = province
    dictjson['city'] = city
    dictjson['city_level'] = city_level
    dictjson['district'] = district
    return dictjson

In [5]:
def iterate_df(latlng,ak_key):
    res = pd.DataFrame([])
    for index,row in tqdm(latlng.iterrows()):
        lng_i = row['lng']
        lat_i = row['lat']
        print(lng_i,lat_i)  # 经纬度
        dictjson = jsonFormat(lat=lat_i,lng=lng_i,ak_key=ak_key) # 通过经纬度调用函数获取位置
        # Add original longitude and latitude as master keys
        dictjson['lng'] = lng_i
        dictjson['lat'] = lat_i
        # Convert dictionary to dataframe
        df_i = pd.DataFrame([dictjson])
        # Concatenate the res
        res = pd.concat([res,df_i])
    # Save the final result
#     res.to_pickle('./res_testlocation.pkl') # PKL
    res.to_excel('./res_testlocation.xlsx',encoding='utf-8-sig') # Excel

In [6]:
if __name__ == '__main__':
    latlng = df[col_lng_lat]
    latlng.columns = ['lng','lat']
    latlng=latlng.astype(str) 
    # print(getlocation(lat, lng))
    # print("------------------")
    # print(jsonFormat(lat,lng))
    iterate_df(latlng,ak_key)

116.366 39.8673
Retrieving:  http://api.map.baidu.com/reverse_geocoding/v3/?ak=lLoGaQmqyIBLGibfmVP9eZRiDGEsseth&output=json&coordtype=wgs84ll&location=39.8673,116.366
117.151 39.097
Retrieving:  http://api.map.baidu.com/reverse_geocoding/v3/?ak=lLoGaQmqyIBLGibfmVP9eZRiDGEsseth&output=json&coordtype=wgs84ll&location=39.097,117.151
nan nan
Retrieving:  http://api.map.baidu.com/reverse_geocoding/v3/?ak=lLoGaQmqyIBLGibfmVP9eZRiDGEsseth&output=json&coordtype=wgs84ll&location=nan,nan
======Failure====
Cannot obtain address 经度： nan ； 纬度:  nan
118.1662 39.6308
Retrieving:  http://api.map.baidu.com/reverse_geocoding/v3/?ak=lLoGaQmqyIBLGibfmVP9eZRiDGEsseth&output=json&coordtype=wgs84ll&location=39.6308,118.1662
119.5259 39.8283
Retrieving:  http://api.map.baidu.com/reverse_geocoding/v3/?ak=lLoGaQmqyIBLGibfmVP9eZRiDGEsseth&output=json&coordtype=wgs84ll&location=39.8283,119.5259

